In [40]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [1]:
cd ..

c:\Projects\python\recommender


In [2]:
from functools import partial
from pathlib import Path

import numpy as np
import pandas as pd
import torch as T
import torch.optim as optim

from utils import get_log_dir

In [3]:
from datasets import SeqTopcoder
from models import FMLearner, TorchFM, TorchHrmFM, TorchPrmeFM, TorchTransFM
from models.fm_learner import simple_loss, trans_loss, simple_weight_loss, trans_weight_loss

In [4]:
DEVICE = T.cuda.current_device()
BATCH = 2000
SHUFFLE = True
WORKERS = 0
NEG_SAMPLE = 5
REGS_PATH = Path("./inputs/topcoder/regs.csv")
CHAG_PATH = Path("./inputs/topcoder/challenge.csv")

In [5]:
db = SeqTopcoder(regs_path=REGS_PATH, chag_path=CHAG_PATH)
db

Read dataset in inputs\topcoder\regs.csv
Original regs shape: (610025, 3)
Original registants size: 60017
Original challenges size: 39916
Filter dataframe shape: (544568, 3)
Index(['challengeId', 'period', 'date', 'prizes', 'technologies', 'platforms'], dtype='object')


In [6]:
db.config_db(batch_size=BATCH,
             shuffle=SHUFFLE,
             num_workers=WORKERS,
             device=DEVICE,
             neg_sample=NEG_SAMPLE)

In [7]:
feat_dim = db.feat_dim
NUM_DIM = 124
INIT_MEAN = 0.1

### Create criterion

In [8]:
# regst setting
LINEAR_REG = 1
EMB_REG = 1
TRANS_REG = 1

In [9]:
simple_loss_callback = partial(simple_loss, LINEAR_REG, EMB_REG)
simple_loss_callback

functools.partial(<function simple_loss at 0x0000020B88782620>, 1, 1)

In [10]:
trans_loss_callback = partial(trans_loss, LINEAR_REG, EMB_REG, TRANS_REG)
trans_loss_callback

functools.partial(<function trans_loss at 0x0000020B887D6840>, 1, 1, 1)

In [11]:
simple_weight_loss_callback = partial(simple_weight_loss, LINEAR_REG, EMB_REG)
simple_weight_loss_callback

functools.partial(<function simple_weight_loss at 0x0000020B887D68C8>, 1, 1)

In [12]:
trans_weight_loss_callback = partial(trans_weight_loss, LINEAR_REG, EMB_REG, TRANS_REG)
trans_weight_loss_callback

functools.partial(<function trans_weight_loss at 0x0000020B887D6950>, 1, 1, 1)

### Train model

#### Hyper-parameter

In [13]:
feat_dim = db.feat_dim
NUM_DIM = 124
INIT_MEAN = 0.1

#### Train FM Model

In [14]:
LEARNING_RATE = 0.001
DECAY_FREQ = 1000
DECAY_GAMMA = 1

In [32]:
fm_model = TorchFM(feature_dim=feat_dim, num_dim=NUM_DIM, init_mean=INIT_MEAN)
fm_model

TorchFM()

In [33]:
adam_opt = optim.Adam(fm_model.parameters(), lr=LEARNING_RATE)
schedular = optim.lr_scheduler.StepLR(adam_opt,
                                      step_size=DECAY_FREQ,
                                      gamma=DECAY_GAMMA)

In [34]:
fm_learner = FMLearner(fm_model, adam_opt, schedular, db)
fm_learner

In [20]:
fm_learner.compile(train_col='base',
                   valid_col='seq',
                   test_col='seq',
                   loss_callback=simple_loss_callback)

In [29]:
fm_learner.compile(train_col='seq',
                   valid_col='seq',
                   test_col='seq',
                   loss_callback=simple_loss_callback)

In [35]:
fm_learner.compile(train_col='seq',
                   valid_col='seq',
                   test_col='seq',
                   loss_callback=simple_weight_loss_callback)

In [30]:
fm_learner.fit(epoch=5,
               log_dir=get_log_dir('simple_topcoder', 'fm'))

  0%|                                                                                                                                                                                     | 0/5 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 46546.154640854336
Epoch 0 step 1: training accuarcy: 0.36760000000000004
Epoch 0 step 1: training loss: 44140.504792487875
Epoch 0 step 2: training accuarcy: 0.4319
Epoch 0 step 2: training loss: 41675.07565998279
Epoch 0 step 3: training accuarcy: 0.5108
Epoch 0 step 3: training loss: 39450.99958670329
Epoch 0 step 4: training accuarcy: 0.5817
Epoch 0 step 4: training loss: 37480.27146773438
Epoch 0 step 5: training accuarcy: 0.6623
Epoch 0 step 5: training loss: 35875.01403513422
Epoch 0 step 6: training accuarcy: 0.7083
Epoch 0 step 6: training loss: 34544.81596775044
Epoch 0 step 7: training accuarcy: 0.7407
Epoch 0 step 7: training loss: 33181.54690224211
Epoch 0 step 8: training accuarcy: 0.7913
Epoch 0 step 8: training loss: 32084.20438950817
Epoch 0 step 9: training accuarcy: 0.8089000000000001
Epoch 0 step 9: training loss: 31093.71356452987
Epoch 0 step 10: training accuarcy: 0.8297
Epoch 0 step 10: training loss: 30231.732255741055
Ep

Epoch 0 step 166: training accuarcy: 0.9862000000000001
Epoch 0 step 166: training loss: 2696.7118351185077
Epoch 0 step 167: training accuarcy: 0.9857
Epoch 0 step 167: training loss: 2681.0778855298404
Epoch 0 step 168: training accuarcy: 0.9868
Epoch 0 step 168: training loss: 2626.978286697358
Epoch 0 step 169: training accuarcy: 0.9872000000000001
Epoch 0 step 169: training loss: 2614.225238317762
Epoch 0 step 170: training accuarcy: 0.9849
Epoch 0 step 170: training loss: 2572.3588066466223
Epoch 0 step 171: training accuarcy: 0.9921000000000001
Epoch 0 step 171: training loss: 2562.2526673500697
Epoch 0 step 172: training accuarcy: 0.9870000000000001
Epoch 0 step 172: training loss: 2502.9905367897736
Epoch 0 step 173: training accuarcy: 0.9887
Epoch 0 step 173: training loss: 2456.2448976557566
Epoch 0 step 174: training accuarcy: 0.9894000000000001
Epoch 0 step 174: training loss: 2546.7299974681673
Epoch 0 step 175: training accuarcy: 0.9840000000000001
Epoch 0 step 175: trai

 20%|██████████████████████████████████▍                                                                                                                                         | 1/5 [04:56<19:45, 296.36s/it]

Epoch: 1
Epoch 1 step 263: training loss: 1141.8540969200897
Epoch 1 step 264: training accuarcy: 0.9913000000000001
Epoch 1 step 264: training loss: 1119.3933070159264
Epoch 1 step 265: training accuarcy: 0.995
Epoch 1 step 265: training loss: 1100.372395073889
Epoch 1 step 266: training accuarcy: 0.9957
Epoch 1 step 266: training loss: 1091.5917523604219
Epoch 1 step 267: training accuarcy: 0.995
Epoch 1 step 267: training loss: 1053.180513373251
Epoch 1 step 268: training accuarcy: 0.9955
Epoch 1 step 268: training loss: 1078.1888165328292
Epoch 1 step 269: training accuarcy: 0.9936
Epoch 1 step 269: training loss: 1075.3834084892223
Epoch 1 step 270: training accuarcy: 0.9936
Epoch 1 step 270: training loss: 1027.8584271406962
Epoch 1 step 271: training accuarcy: 0.9966
Epoch 1 step 271: training loss: 1046.4623134771666
Epoch 1 step 272: training accuarcy: 0.9955
Epoch 1 step 272: training loss: 1028.8387499158332
Epoch 1 step 273: training accuarcy: 0.9951000000000001
Epoch 1 ste

...

Epoch: 2
Epoch 2 step 526: training loss: 383.67212274899856
Epoch 2 step 527: training accuarcy: 0.9966
Epoch 2 step 527: training loss: 384.8049289700171
Epoch 2 step 528: training accuarcy: 0.9978
Epoch 2 step 528: training loss: 356.31919884529856
Epoch 2 step 529: training accuarcy: 0.9978
Epoch 2 step 529: training loss: 356.761232820384
Epoch 2 step 530: training accuarcy: 0.999
Epoch 2 step 530: training loss: 362.7373632910302
Epoch 2 step 531: training accuarcy: 0.9981000000000001
Epoch 2 step 531: training loss: 409.8708334306421
Epoch 2 step 532: training accuarcy: 0.9961000000000001
Epoch 2 step 532: training loss: 370.6629892123538
Epoch 2 step 533: training accuarcy: 0.9965
Epoch 2 step 533: training loss: 355.17098244522646
Epoch 2 step 534: training accuarcy: 0.999
Epoch 2 step 534: training loss: 378.93532783609265
Epoch 2 step 535: training accuarcy: 0.9966
Epoch 2 step 535: training loss: 339.88251935452354
Epoch 2 step 536: training accuarcy: 0.9983000000000001
Epo

Epoch 2 step 688: training loss: 375.2483641722556
Epoch 2 step 689: training accuarcy: 0.9983000000000001
Epoch 2 step 689: training loss: 342.9448350660174
Epoch 2 step 690: training accuarcy: 0.9979
Epoch 2 step 690: training loss: 359.4577075171537
Epoch 2 step 691: training accuarcy: 0.9951000000000001
Epoch 2 step 691: training loss: 330.59003584137616
Epoch 2 step 692: training accuarcy: 0.9986
Epoch 2 step 692: training loss: 340.5669023189479
Epoch 2 step 693: training accuarcy: 0.9961000000000001
Epoch 2 step 693: training loss: 350.6996320764523
Epoch 2 step 694: training accuarcy: 0.9964000000000001
Epoch 2 step 694: training loss: 374.09681592900597
Epoch 2 step 695: training accuarcy: 0.9971000000000001
Epoch 2 step 695: training loss: 336.74055270371014
Epoch 2 step 696: training accuarcy: 0.9966
Epoch 2 step 696: training loss: 363.0581063136573
Epoch 2 step 697: training accuarcy: 0.996
Epoch 2 step 697: training loss: 353.8461701288368
Epoch 2 step 698: training accua

 60%|███████████████████████████████████████████████████████████████████████████████████████████████████████▏                                                                    | 3/5 [14:28<09:41, 290.78s/it]

Epoch: 3
Epoch 3 step 789: training loss: 296.23032878304855
Epoch 3 step 790: training accuarcy: 0.999
Epoch 3 step 790: training loss: 300.7896643394858
Epoch 3 step 791: training accuarcy: 0.9983000000000001
Epoch 3 step 791: training loss: 293.4835120560487
Epoch 3 step 792: training accuarcy: 0.9989
Epoch 3 step 792: training loss: 285.5181151599255
Epoch 3 step 793: training accuarcy: 0.9991000000000001
Epoch 3 step 793: training loss: 302.478868166378
Epoch 3 step 794: training accuarcy: 0.9982000000000001
Epoch 3 step 794: training loss: 316.27082484827275
Epoch 3 step 795: training accuarcy: 0.9989
Epoch 3 step 795: training loss: 293.5194634860173
Epoch 3 step 796: training accuarcy: 0.9977
Epoch 3 step 796: training loss: 298.44195453152906
Epoch 3 step 797: training accuarcy: 0.9985
Epoch 3 step 797: training loss: 274.5905129792551
Epoch 3 step 798: training accuarcy: 0.9989
Epoch 3 step 798: training loss: 313.0679676110951
Epoch 3 step 799: training accuarcy: 0.9989
Epoc

KeyboardInterrupt: 

In [36]:
fm_learner.fit(epoch=3, 
               log_dir=get_log_dir("weight_topcoder", "fm"))


  0%|                                                                                                                                                                                     | 0/3 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 96099.51260676965
Epoch 0 step 1: training accuarcy: 0.29510000000000003
Epoch 0 step 1: training loss: 88335.88527898806
Epoch 0 step 2: training accuarcy: 0.3618
Epoch 0 step 2: training loss: 82955.90096143921
Epoch 0 step 3: training accuarcy: 0.41050000000000003
Epoch 0 step 3: training loss: 75841.65049929358
Epoch 0 step 4: training accuarcy: 0.47500000000000003
Epoch 0 step 4: training loss: 69900.84115374796
Epoch 0 step 5: training accuarcy: 0.5316000000000001
Epoch 0 step 5: training loss: 66515.16308581155
Epoch 0 step 6: training accuarcy: 0.5718
Epoch 0 step 6: training loss: 62505.37561871747
Epoch 0 step 7: training accuarcy: 0.6056
Epoch 0 step 7: training loss: 57537.948671329985
Epoch 0 step 8: training accuarcy: 0.6546000000000001
Epoch 0 step 8: training loss: 56306.74673244863
Epoch 0 step 9: training accuarcy: 0.6639
Epoch 0 step 9: training loss: 54004.51565162408
Epoch 0 step 10: training accuarcy: 0.6841
Epoch 0 step 10:

Epoch 0 step 166: training accuarcy: 0.9649000000000001
Epoch 0 step 166: training loss: 12025.231994584245
Epoch 0 step 167: training accuarcy: 0.9740000000000001
Epoch 0 step 167: training loss: 12975.623940105927
Epoch 0 step 168: training accuarcy: 0.9597
Epoch 0 step 168: training loss: 12528.237092144336
Epoch 0 step 169: training accuarcy: 0.9664
Epoch 0 step 169: training loss: 12116.927013772427
Epoch 0 step 170: training accuarcy: 0.9694
Epoch 0 step 170: training loss: 12190.54159559381
Epoch 0 step 171: training accuarcy: 0.9705
Epoch 0 step 171: training loss: 11859.6976563459
Epoch 0 step 172: training accuarcy: 0.9723
Epoch 0 step 172: training loss: 12199.927979699658
Epoch 0 step 173: training accuarcy: 0.9687
Epoch 0 step 173: training loss: 12409.036370485526
Epoch 0 step 174: training accuarcy: 0.9647
Epoch 0 step 174: training loss: 11917.692596284536
Epoch 0 step 175: training accuarcy: 0.9752000000000001
Epoch 0 step 175: training loss: 12528.170015238262
Epoch 0


 33%|█████████████████████████████████████████████████████████▎                                                                                                                  | 1/3 [04:50<09:41, 290.77s/it]

Epoch: 1
Epoch 1 step 263: training loss: 8943.806104887635
Epoch 1 step 264: training accuarcy: 0.9852000000000001
Epoch 1 step 264: training loss: 8739.16893308108
Epoch 1 step 265: training accuarcy: 0.9881000000000001
Epoch 1 step 265: training loss: 8939.825616020917
Epoch 1 step 266: training accuarcy: 0.9856
Epoch 1 step 266: training loss: 8735.961095957951
Epoch 1 step 267: training accuarcy: 0.9875
Epoch 1 step 267: training loss: 8881.61574397262
Epoch 1 step 268: training accuarcy: 0.9870000000000001
Epoch 1 step 268: training loss: 8927.158958352806
Epoch 1 step 269: training accuarcy: 0.9848
Epoch 1 step 269: training loss: 8834.69927256897
Epoch 1 step 270: training accuarcy: 0.9830000000000001
Epoch 1 step 270: training loss: 8841.092411649668
Epoch 1 step 271: training accuarcy: 0.9851000000000001
Epoch 1 step 271: training loss: 8631.794868342997
Epoch 1 step 272: training accuarcy: 0.9862000000000001
Epoch 1 step 272: training loss: 8458.694663804956
Epoch 1 step 273

Epoch 1 step 426: training loss: 6221.554330038238
Epoch 1 step 427: training accuarcy: 0.9876
Epoch 1 step 427: training loss: 5927.4489242996915
Epoch 1 step 428: training accuarcy: 0.9934000000000001
Epoch 1 step 428: training loss: 6041.6842210320665
Epoch 1 step 429: training accuarcy: 0.9885
Epoch 1 step 429: training loss: 5944.180503888112
Epoch 1 step 430: training accuarcy: 0.9895
Epoch 1 step 430: training loss: 5981.051606968576
Epoch 1 step 431: training accuarcy: 0.9901000000000001
Epoch 1 step 431: training loss: 5886.426304914507
Epoch 1 step 432: training accuarcy: 0.9914000000000001
Epoch 1 step 432: training loss: 5959.966622372545
Epoch 1 step 433: training accuarcy: 0.9907
Epoch 1 step 433: training loss: 5846.8005121209135
Epoch 1 step 434: training accuarcy: 0.9912000000000001
Epoch 1 step 434: training loss: 5879.018690676422
Epoch 1 step 435: training accuarcy: 0.9896
Epoch 1 step 435: training loss: 5900.992912393967
Epoch 1 step 436: training accuarcy: 0.9900


 67%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                                                         | 2/3 [09:35<04:49, 289.03s/it]

Epoch: 2
Epoch 2 step 526: training loss: 4345.961915111372
Epoch 2 step 527: training accuarcy: 0.9978
Epoch 2 step 527: training loss: 4260.850086770885
Epoch 2 step 528: training accuarcy: 0.9973000000000001
Epoch 2 step 528: training loss: 4487.605909708647
Epoch 2 step 529: training accuarcy: 0.9956
Epoch 2 step 529: training loss: 4215.843818764394
Epoch 2 step 530: training accuarcy: 0.9966
Epoch 2 step 530: training loss: 4288.54462929906
Epoch 2 step 531: training accuarcy: 0.9954000000000001
Epoch 2 step 531: training loss: 4238.23354934376
Epoch 2 step 532: training accuarcy: 0.9973000000000001
Epoch 2 step 532: training loss: 4265.6264095508295
Epoch 2 step 533: training accuarcy: 0.997
Epoch 2 step 533: training loss: 4356.499136132832
Epoch 2 step 534: training accuarcy: 0.9961000000000001
Epoch 2 step 534: training loss: 4433.148706856522
Epoch 2 step 535: training accuarcy: 0.9942000000000001
Epoch 2 step 535: training loss: 4427.379311821591
Epoch 2 step 536: training 

Epoch 2 step 689: training accuarcy: 0.9943000000000001
Epoch 2 step 689: training loss: 3416.9060598718124
Epoch 2 step 690: training accuarcy: 0.9939
Epoch 2 step 690: training loss: 3396.018059200225
Epoch 2 step 691: training accuarcy: 0.994
Epoch 2 step 691: training loss: 3282.170893567474
Epoch 2 step 692: training accuarcy: 0.9951000000000001
Epoch 2 step 692: training loss: 3317.985513265985
Epoch 2 step 693: training accuarcy: 0.9928
Epoch 2 step 693: training loss: 3234.8365542682873
Epoch 2 step 694: training accuarcy: 0.9958
Epoch 2 step 694: training loss: 3220.3541611956457
Epoch 2 step 695: training accuarcy: 0.9939
Epoch 2 step 695: training loss: 3209.0935824468315
Epoch 2 step 696: training accuarcy: 0.995
Epoch 2 step 696: training loss: 3167.241672729309
Epoch 2 step 697: training accuarcy: 0.997
Epoch 2 step 697: training loss: 3224.179567806962
Epoch 2 step 698: training accuarcy: 0.9953000000000001
Epoch 2 step 698: training loss: 3485.1692240131806
Epoch 2 step


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [14:23<00:00, 288.59s/it]

In [31]:
del fm_model
T.cuda.empty_cache()

### Train HRM FM Model

In [21]:
hrm_model = TorchHrmFM(feature_dim=feat_dim, num_dim=NUM_DIM, init_mean=INIT_MEAN)
hrm_model

TorchHrmFM()

In [22]:
adam_opt = optim.Adam(hrm_model.parameters(), lr=LEARNING_RATE)
schedular = optim.lr_scheduler.StepLR(adam_opt,
                                      step_size=DECAY_FREQ,
                                      gamma=DECAY_GAMMA)

In [23]:
hrm_learner = FMLearner(hrm_model, adam_opt, schedular, db)
hrm_learner

In [16]:
hrm_learner.compile(train_col='base',
                   valid_col='seq',
                   test_col='seq',
                   loss_callback=simple_loss_callback)

In [18]:
hrm_learner.compile(train_col='seq',
                    valid_col='seq',
                    test_col='seq',
                    loss_callback=simple_loss_callback)

In [24]:
hrm_learner.compile(train_col='seq',
                    valid_col='seq',
                    test_col='seq',
                    loss_callback=simple_weight_loss_callback)

In [19]:
hrm_learner.fit(epoch=3,
                log_dir=get_log_dir('simple_topcoder', 'hrm'))

  0%|                                                     | 0/3 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 67269.71110027473
Epoch 0 step 1: training accuarcy: 0.505
Epoch 0 step 1: training loss: 64582.12846184061
Epoch 0 step 2: training accuarcy: 0.5214
Epoch 0 step 2: training loss: 62127.96202200205
Epoch 0 step 3: training accuarcy: 0.5177
Epoch 0 step 3: training loss: 60768.18910236
Epoch 0 step 4: training accuarcy: 0.5222
Epoch 0 step 4: training loss: 60158.479771239865
Epoch 0 step 5: training accuarcy: 0.5176000000000001
Epoch 0 step 5: training loss: 58003.848322113045
Epoch 0 step 6: training accuarcy: 0.5195000000000001
Epoch 0 step 6: training loss: 56235.06773408047
Epoch 0 step 7: training accuarcy: 0.529
Epoch 0 step 7: training loss: 54929.95489985055
Epoch 0 step 8: training accuarcy: 0.5187
Epoch 0 step 8: training loss: 53652.64739050824
Epoch 0 step 9: training accuarcy: 0.5205000000000001
Epoch 0 step 9: training loss: 52567.50304010458
Epoch 0 step 10: training accuarcy: 0.52
Epoch 0 step 10: training loss: 51398.31691935751

Epoch 0 step 85: training loss: 11096.330334479437
Epoch 0 step 86: training accuarcy: 0.5952000000000001
Epoch 0 step 86: training loss: 10942.775706173958
Epoch 0 step 87: training accuarcy: 0.5995
Epoch 0 step 87: training loss: 10942.955861821483
Epoch 0 step 88: training accuarcy: 0.5996
Epoch 0 step 88: training loss: 10831.690345903531
Epoch 0 step 89: training accuarcy: 0.6018
Epoch 0 step 89: training loss: 10771.428398885564
Epoch 0 step 90: training accuarcy: 0.5910000000000001
Epoch 0 step 90: training loss: 10587.251798078165
Epoch 0 step 91: training accuarcy: 0.6021000000000001
Epoch 0 step 91: training loss: 10574.731046866284
Epoch 0 step 92: training accuarcy: 0.6083000000000001
Epoch 0 step 92: training loss: 10456.53724118756
Epoch 0 step 93: training accuarcy: 0.6053000000000001
Epoch 0 step 93: training loss: 10391.278152943694
Epoch 0 step 94: training accuarcy: 0.5991000000000001
Epoch 0 step 94: training loss: 10311.247992718909
Epoch 0 step 95: training accuar

Epoch 0 step 168: training accuarcy: 0.6943
Epoch 0 step 168: training loss: 7368.357885820251
Epoch 0 step 169: training accuarcy: 0.6969000000000001
Epoch 0 step 169: training loss: 7305.313833381052
Epoch 0 step 170: training accuarcy: 0.6973
Epoch 0 step 170: training loss: 7218.462005929325
Epoch 0 step 171: training accuarcy: 0.7048
Epoch 0 step 171: training loss: 7362.048658092097
Epoch 0 step 172: training accuarcy: 0.6807000000000001
Epoch 0 step 172: training loss: 7287.953607258028
Epoch 0 step 173: training accuarcy: 0.6861
Epoch 0 step 173: training loss: 7179.1870674379525
Epoch 0 step 174: training accuarcy: 0.7009000000000001
Epoch 0 step 174: training loss: 7143.729533500957
Epoch 0 step 175: training accuarcy: 0.7035
Epoch 0 step 175: training loss: 7166.902411097375
Epoch 0 step 176: training accuarcy: 0.6954
Epoch 0 step 176: training loss: 7129.452412200186
Epoch 0 step 177: training accuarcy: 0.7050000000000001
Epoch 0 step 177: training loss: 7172.740418741923
E

Epoch 0 step 251: training accuarcy: 0.7265
Epoch 0 step 251: training loss: 6444.195912060558
Epoch 0 step 252: training accuarcy: 0.7342000000000001
Epoch 0 step 252: training loss: 6471.524739626552
Epoch 0 step 253: training accuarcy: 0.7113
Epoch 0 step 253: training loss: 6352.942340754675
Epoch 0 step 254: training accuarcy: 0.728
Epoch 0 step 254: training loss: 6440.121396515456
Epoch 0 step 255: training accuarcy: 0.7275
Epoch 0 step 255: training loss: 6394.942488010187
Epoch 0 step 256: training accuarcy: 0.7234
Epoch 0 step 256: training loss: 6440.387036586294
Epoch 0 step 257: training accuarcy: 0.7189
Epoch 0 step 257: training loss: 6441.134515679929
Epoch 0 step 258: training accuarcy: 0.7188
Epoch 0 step 258: training loss: 6436.9168380502615
Epoch 0 step 259: training accuarcy: 0.7164
Epoch 0 step 259: training loss: 6428.164205314782
Epoch 0 step 260: training accuarcy: 0.7286
Epoch 0 step 260: training loss: 6357.88764939358
Epoch 0 step 261: training accuarcy: 0.

...

Epoch: 2
Epoch 2 step 526: training loss: 5834.380269160445
Epoch 2 step 527: training accuarcy: 0.7543000000000001
Epoch 2 step 527: training loss: 5891.897092834732
Epoch 2 step 528: training accuarcy: 0.7492000000000001
Epoch 2 step 528: training loss: 5894.3913743412995
Epoch 2 step 529: training accuarcy: 0.7562
Epoch 2 step 529: training loss: 5927.385883920442
Epoch 2 step 530: training accuarcy: 0.7477
Epoch 2 step 530: training loss: 5895.924112599176
Epoch 2 step 531: training accuarcy: 0.7507
Epoch 2 step 531: training loss: 5918.430441360658
Epoch 2 step 532: training accuarcy: 0.7543000000000001
Epoch 2 step 532: training loss: 5936.418458656189
Epoch 2 step 533: training accuarcy: 0.7459
Epoch 2 step 533: training loss: 5886.256474486059
Epoch 2 step 534: training accuarcy: 0.7557
Epoch 2 step 534: training loss: 5894.4572986412895
Epoch 2 step 535: training accuarcy: 0.7393000000000001
Epoch 2 step 535: training loss: 5886.154137588903
Epoch 2 step 536: training accuarcy

Epoch 2 step 609: training loss: 5895.3893517752085
Epoch 2 step 610: training accuarcy: 0.7539
Epoch 2 step 610: training loss: 5830.257277635872
Epoch 2 step 611: training accuarcy: 0.7564000000000001
Epoch 2 step 611: training loss: 5923.443772707324
Epoch 2 step 612: training accuarcy: 0.7369
Epoch 2 step 612: training loss: 5900.989644823518
Epoch 2 step 613: training accuarcy: 0.7384000000000001
Epoch 2 step 613: training loss: 5900.434202490933
Epoch 2 step 614: training accuarcy: 0.7432000000000001
Epoch 2 step 614: training loss: 5816.797334856353
Epoch 2 step 615: training accuarcy: 0.7524000000000001
Epoch 2 step 615: training loss: 5866.640021173241
Epoch 2 step 616: training accuarcy: 0.7551
Epoch 2 step 616: training loss: 5854.442000147157
Epoch 2 step 617: training accuarcy: 0.7437
Epoch 2 step 617: training loss: 5822.733543877129
Epoch 2 step 618: training accuarcy: 0.7572
Epoch 2 step 618: training loss: 5841.33672958609
Epoch 2 step 619: training accuarcy: 0.7505000

Epoch 2 step 692: training loss: 5811.835020194658
Epoch 2 step 693: training accuarcy: 0.7552
Epoch 2 step 693: training loss: 5883.713707461016
Epoch 2 step 694: training accuarcy: 0.738
Epoch 2 step 694: training loss: 5907.087331879984
Epoch 2 step 695: training accuarcy: 0.7435
Epoch 2 step 695: training loss: 5846.979468365015
Epoch 2 step 696: training accuarcy: 0.743
Epoch 2 step 696: training loss: 5817.106911084528
Epoch 2 step 697: training accuarcy: 0.7534000000000001
Epoch 2 step 697: training loss: 5791.586188503723
Epoch 2 step 698: training accuarcy: 0.7467
Epoch 2 step 698: training loss: 5867.808732984938
Epoch 2 step 699: training accuarcy: 0.751
Epoch 2 step 699: training loss: 5850.6833622012555
Epoch 2 step 700: training accuarcy: 0.7494000000000001
Epoch 2 step 700: training loss: 5849.148307561865
Epoch 2 step 701: training accuarcy: 0.7473000000000001
Epoch 2 step 701: training loss: 5875.796910377224
Epoch 2 step 702: training accuarcy: 0.7435
Epoch 2 step 702

Epoch 2 step 775: training loss: 5890.86072994669
Epoch 2 step 776: training accuarcy: 0.7434000000000001
Epoch 2 step 776: training loss: 5846.890042634417
Epoch 2 step 777: training accuarcy: 0.7473000000000001
Epoch 2 step 777: training loss: 5765.618175042394
Epoch 2 step 778: training accuarcy: 0.7655000000000001
Epoch 2 step 778: training loss: 5775.844352081733
Epoch 2 step 779: training accuarcy: 0.7468
Epoch 2 step 779: training loss: 5842.100830162649
Epoch 2 step 780: training accuarcy: 0.7516
Epoch 2 step 780: training loss: 5750.1457929048875
Epoch 2 step 781: training accuarcy: 0.7579
Epoch 2 step 781: training loss: 5909.569845082015
Epoch 2 step 782: training accuarcy: 0.7381000000000001
Epoch 2 step 782: training loss: 5763.105566623627
Epoch 2 step 783: training accuarcy: 0.7535000000000001
Epoch 2 step 783: training loss: 5861.269826782854
Epoch 2 step 784: training accuarcy: 0.7473000000000001
Epoch 2 step 784: training loss: 5780.871625873077
Epoch 2 step 785: trai

100%|████████████████████████████████████████████| 3/3 [15:11<00:00, 303.90s/it]


In [25]:
hrm_learner.fit(epoch=3,
                log_dir=get_log_dir('weight_topcoder', 'hrm'))

  0%|                                                     | 0/3 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 193365.16163974453
Epoch 0 step 1: training accuarcy: 0.51
Epoch 0 step 1: training loss: 185119.86205282228
Epoch 0 step 2: training accuarcy: 0.5207
Epoch 0 step 2: training loss: 189420.40994518064
Epoch 0 step 3: training accuarcy: 0.5195000000000001
Epoch 0 step 3: training loss: 179850.18585890112
Epoch 0 step 4: training accuarcy: 0.5253
Epoch 0 step 4: training loss: 176592.3830378908
Epoch 0 step 5: training accuarcy: 0.5118
Epoch 0 step 5: training loss: 171351.80347387
Epoch 0 step 6: training accuarcy: 0.5213
Epoch 0 step 6: training loss: 164100.8087699164
Epoch 0 step 7: training accuarcy: 0.5284
Epoch 0 step 7: training loss: 161553.85251411583
Epoch 0 step 8: training accuarcy: 0.5347000000000001
Epoch 0 step 8: training loss: 162316.5572234422
Epoch 0 step 9: training accuarcy: 0.5323
Epoch 0 step 9: training loss: 153082.98121592737
Epoch 0 step 10: training accuarcy: 0.522
Epoch 0 step 10: training loss: 150206.3848270157
Epoch

Epoch 0 step 86: training accuarcy: 0.5819
Epoch 0 step 86: training loss: 44889.39903795921
Epoch 0 step 87: training accuarcy: 0.5967
Epoch 0 step 87: training loss: 44594.90324295683
Epoch 0 step 88: training accuarcy: 0.6064
Epoch 0 step 88: training loss: 44248.53584450766
Epoch 0 step 89: training accuarcy: 0.607
Epoch 0 step 89: training loss: 44968.39339135873
Epoch 0 step 90: training accuarcy: 0.5903
Epoch 0 step 90: training loss: 44462.97636799871
Epoch 0 step 91: training accuarcy: 0.5972000000000001
Epoch 0 step 91: training loss: 43692.35513216111
Epoch 0 step 92: training accuarcy: 0.6045
Epoch 0 step 92: training loss: 43768.13933397746
Epoch 0 step 93: training accuarcy: 0.6159
Epoch 0 step 93: training loss: 44247.50242781735
Epoch 0 step 94: training accuarcy: 0.5985
Epoch 0 step 94: training loss: 43860.23705441014
Epoch 0 step 95: training accuarcy: 0.612
Epoch 0 step 95: training loss: 44254.405241494256
Epoch 0 step 96: training accuarcy: 0.5932000000000001
Epoc

Epoch 0 step 169: training loss: 37197.22417312359
Epoch 0 step 170: training accuarcy: 0.6948000000000001
Epoch 0 step 170: training loss: 37271.32799917286
Epoch 0 step 171: training accuarcy: 0.6865
Epoch 0 step 171: training loss: 38013.06343429402
Epoch 0 step 172: training accuarcy: 0.6849000000000001
Epoch 0 step 172: training loss: 37532.25739286332
Epoch 0 step 173: training accuarcy: 0.6824
Epoch 0 step 173: training loss: 37072.33954684183
Epoch 0 step 174: training accuarcy: 0.6904
Epoch 0 step 174: training loss: 36897.478919225105
Epoch 0 step 175: training accuarcy: 0.6941
Epoch 0 step 175: training loss: 36839.429808392735
Epoch 0 step 176: training accuarcy: 0.6974
Epoch 0 step 176: training loss: 36865.56303404251
Epoch 0 step 177: training accuarcy: 0.6892
Epoch 0 step 177: training loss: 37232.450899596624
Epoch 0 step 178: training accuarcy: 0.6881
Epoch 0 step 178: training loss: 36279.362217262074
Epoch 0 step 179: training accuarcy: 0.7096
Epoch 0 step 179: trai

Epoch 0 step 252: training loss: 34278.952959497314
Epoch 0 step 253: training accuarcy: 0.7257
Epoch 0 step 253: training loss: 33501.52451137232
Epoch 0 step 254: training accuarcy: 0.7494000000000001
Epoch 0 step 254: training loss: 33864.68539032806
Epoch 0 step 255: training accuarcy: 0.7278
Epoch 0 step 255: training loss: 34020.38493017855
Epoch 0 step 256: training accuarcy: 0.7354
Epoch 0 step 256: training loss: 34341.72044269684
Epoch 0 step 257: training accuarcy: 0.7245
Epoch 0 step 257: training loss: 33554.52557347421
Epoch 0 step 258: training accuarcy: 0.7383000000000001
Epoch 0 step 258: training loss: 33889.85953733517
Epoch 0 step 259: training accuarcy: 0.736
Epoch 0 step 259: training loss: 34254.14105188148
Epoch 0 step 260: training accuarcy: 0.7288
Epoch 0 step 260: training loss: 34180.93643714745
Epoch 0 step 261: training accuarcy: 0.7249
Epoch 0 step 261: training loss: 33865.30062586484
Epoch 0 step 262: training accuarcy: 0.7386
Epoch 0 step 262: training

...

Epoch: 2
Epoch 2 step 526: training loss: 29444.827389827682
Epoch 2 step 527: training accuarcy: 0.7862
Epoch 2 step 527: training loss: 28523.125714488488
Epoch 2 step 528: training accuarcy: 0.8022
Epoch 2 step 528: training loss: 29318.442319866088
Epoch 2 step 529: training accuarcy: 0.7877000000000001
Epoch 2 step 529: training loss: 29288.732426444403
Epoch 2 step 530: training accuarcy: 0.7866000000000001
Epoch 2 step 530: training loss: 28845.008535668574
Epoch 2 step 531: training accuarcy: 0.7902
Epoch 2 step 531: training loss: 29109.83316338543
Epoch 2 step 532: training accuarcy: 0.7919
Epoch 2 step 532: training loss: 29101.957448656427
Epoch 2 step 533: training accuarcy: 0.79
Epoch 2 step 533: training loss: 29421.874596014342
Epoch 2 step 534: training accuarcy: 0.7853
Epoch 2 step 534: training loss: 29064.878122278165
Epoch 2 step 535: training accuarcy: 0.7874
Epoch 2 step 535: training loss: 29181.4835376725
Epoch 2 step 536: training accuarcy: 0.7942
Epoch 2 step

Epoch 2 step 609: training accuarcy: 0.7785000000000001
Epoch 2 step 609: training loss: 29393.46329736679
Epoch 2 step 610: training accuarcy: 0.7835000000000001
Epoch 2 step 610: training loss: 29467.41881039033
Epoch 2 step 611: training accuarcy: 0.778
Epoch 2 step 611: training loss: 29608.2089999082
Epoch 2 step 612: training accuarcy: 0.7785000000000001
Epoch 2 step 612: training loss: 29824.281984113375
Epoch 2 step 613: training accuarcy: 0.7704000000000001
Epoch 2 step 613: training loss: 28741.93647236604
Epoch 2 step 614: training accuarcy: 0.786
Epoch 2 step 614: training loss: 29166.909862043896
Epoch 2 step 615: training accuarcy: 0.7808
Epoch 2 step 615: training loss: 29332.23251420685
Epoch 2 step 616: training accuarcy: 0.7855000000000001
Epoch 2 step 616: training loss: 29218.20570179459
Epoch 2 step 617: training accuarcy: 0.7796000000000001
Epoch 2 step 617: training loss: 29429.351695108195
Epoch 2 step 618: training accuarcy: 0.7812
Epoch 2 step 618: training lo

Epoch 2 step 690: training loss: 28597.40305173381
Epoch 2 step 691: training accuarcy: 0.7861
Epoch 2 step 691: training loss: 28287.801654713963
Epoch 2 step 692: training accuarcy: 0.7956000000000001
Epoch 2 step 692: training loss: 29244.290852748567
Epoch 2 step 693: training accuarcy: 0.7778
Epoch 2 step 693: training loss: 28655.785406814524
Epoch 2 step 694: training accuarcy: 0.7754000000000001
Epoch 2 step 694: training loss: 28746.35303665627
Epoch 2 step 695: training accuarcy: 0.7852
Epoch 2 step 695: training loss: 28817.037135293744
Epoch 2 step 696: training accuarcy: 0.7873
Epoch 2 step 696: training loss: 28885.19211856198
Epoch 2 step 697: training accuarcy: 0.7854
Epoch 2 step 697: training loss: 28467.659738975814
Epoch 2 step 698: training accuarcy: 0.7874
Epoch 2 step 698: training loss: 29266.5728954465
Epoch 2 step 699: training accuarcy: 0.7695000000000001
Epoch 2 step 699: training loss: 28353.769339563405
Epoch 2 step 700: training accuarcy: 0.7932
Epoch 2 s

Epoch 2 step 772: training loss: 29019.41296146404
Epoch 2 step 773: training accuarcy: 0.7732
Epoch 2 step 773: training loss: 28652.311985166358
Epoch 2 step 774: training accuarcy: 0.7826000000000001
Epoch 2 step 774: training loss: 28928.243440792026
Epoch 2 step 775: training accuarcy: 0.7745000000000001
Epoch 2 step 775: training loss: 28051.80545566762
Epoch 2 step 776: training accuarcy: 0.7965
Epoch 2 step 776: training loss: 28278.83419560922
Epoch 2 step 777: training accuarcy: 0.7818
Epoch 2 step 777: training loss: 28967.560284302526
Epoch 2 step 778: training accuarcy: 0.7825000000000001
Epoch 2 step 778: training loss: 28562.803906150028
Epoch 2 step 779: training accuarcy: 0.7824
Epoch 2 step 779: training loss: 28608.27201409693
Epoch 2 step 780: training accuarcy: 0.7795000000000001
Epoch 2 step 780: training loss: 28173.920706854355
Epoch 2 step 781: training accuarcy: 0.791
Epoch 2 step 781: training loss: 29469.059511362255
Epoch 2 step 782: training accuarcy: 0.76

100%|████████████████████████████████████████████| 3/3 [15:15<00:00, 304.55s/it]


In [30]:
del hrm_model
T.cuda.empty_cache()

NameError: name 'hrm_model' is not defined

### Train PRME FM Model

In [37]:
prme_model = TorchPrmeFM(feature_dim=feat_dim, num_dim=NUM_DIM, init_mean=INIT_MEAN)
prme_model

TorchPrmeFM()

In [38]:
adam_opt = optim.Adam(prme_model.parameters(), lr=LEARNING_RATE)
schedular = optim.lr_scheduler.StepLR(adam_opt,
                                      step_size=DECAY_FREQ,
                                      gamma=DECAY_GAMMA)

In [39]:
prme_learner = FMLearner(prme_model, adam_opt, schedular, db)
prme_learner

In [22]:
prme_learner.compile(train_col='base',
                   valid_col='seq',
                   test_col='seq',
                   loss_callback=simple_loss_callback)

In [34]:
prme_learner.compile(train_col='seq',
                     valid_col='seq',
                     test_col='seq',
                     loss_callback=simple_loss_callback)

In [40]:
prme_learner.compile(train_col='seq',
                     valid_col='seq',
                     test_col='seq',
                     loss_callback=simple_weight_loss_callback)

In [35]:
prme_learner.fit(epoch=3,
                 log_dir=get_log_dir('simple_topcoder', 'prme'))

  0%|                                                     | 0/3 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 87517.35232690867
Epoch 0 step 1: training accuarcy: 0.5117
Epoch 0 step 1: training loss: 83730.58049910842
Epoch 0 step 2: training accuarcy: 0.5198
Epoch 0 step 2: training loss: 82299.59224676236
Epoch 0 step 3: training accuarcy: 0.5243
Epoch 0 step 3: training loss: 80546.39131474958
Epoch 0 step 4: training accuarcy: 0.5163
Epoch 0 step 4: training loss: 78172.21529430218
Epoch 0 step 5: training accuarcy: 0.5307000000000001
Epoch 0 step 5: training loss: 76259.36374719077
Epoch 0 step 6: training accuarcy: 0.524
Epoch 0 step 6: training loss: 75570.69756017407
Epoch 0 step 7: training accuarcy: 0.5138
Epoch 0 step 7: training loss: 73277.05738821287
Epoch 0 step 8: training accuarcy: 0.5251
Epoch 0 step 8: training loss: 72495.04433095024
Epoch 0 step 9: training accuarcy: 0.5235000000000001
Epoch 0 step 9: training loss: 71819.02370920154
Epoch 0 step 10: training accuarcy: 0.5094000000000001
Epoch 0 step 10: training loss: 69082.1803479

Epoch 0 step 85: training loss: 13646.205122513511
Epoch 0 step 86: training accuarcy: 0.5923
Epoch 0 step 86: training loss: 13544.976658736261
Epoch 0 step 87: training accuarcy: 0.5948
Epoch 0 step 87: training loss: 13313.003614547353
Epoch 0 step 88: training accuarcy: 0.6074
Epoch 0 step 88: training loss: 13213.417451061403
Epoch 0 step 89: training accuarcy: 0.5946
Epoch 0 step 89: training loss: 13196.37139699827
Epoch 0 step 90: training accuarcy: 0.6019
Epoch 0 step 90: training loss: 13278.49043070666
Epoch 0 step 91: training accuarcy: 0.5785
Epoch 0 step 91: training loss: 12814.977529725827
Epoch 0 step 92: training accuarcy: 0.6094
Epoch 0 step 92: training loss: 12860.3565593419
Epoch 0 step 93: training accuarcy: 0.5952000000000001
Epoch 0 step 93: training loss: 12706.35801854423
Epoch 0 step 94: training accuarcy: 0.6013000000000001
Epoch 0 step 94: training loss: 12666.417241812676
Epoch 0 step 95: training accuarcy: 0.6029
Epoch 0 step 95: training loss: 12544.482

Epoch 0 step 169: training accuarcy: 0.6842
Epoch 0 step 169: training loss: 8733.840265688857
Epoch 0 step 170: training accuarcy: 0.6779000000000001
Epoch 0 step 170: training loss: 8581.667510698779
Epoch 0 step 171: training accuarcy: 0.682
Epoch 0 step 171: training loss: 8596.243670823784
Epoch 0 step 172: training accuarcy: 0.6866
Epoch 0 step 172: training loss: 8394.18079441406
Epoch 0 step 173: training accuarcy: 0.6979000000000001
Epoch 0 step 173: training loss: 8453.858895602505
Epoch 0 step 174: training accuarcy: 0.6916
Epoch 0 step 174: training loss: 8413.480537686512
Epoch 0 step 175: training accuarcy: 0.6936
Epoch 0 step 175: training loss: 8403.545066030718
Epoch 0 step 176: training accuarcy: 0.6989000000000001
Epoch 0 step 176: training loss: 8468.05392176198
Epoch 0 step 177: training accuarcy: 0.6851
Epoch 0 step 177: training loss: 8398.80120811788
Epoch 0 step 178: training accuarcy: 0.6916
Epoch 0 step 178: training loss: 8370.494028797255
Epoch 0 step 179: 

Epoch 0 step 251: training loss: 7070.205044751962
Epoch 0 step 252: training accuarcy: 0.735
Epoch 0 step 252: training loss: 7123.175271988205
Epoch 0 step 253: training accuarcy: 0.7287
Epoch 0 step 253: training loss: 7049.062276029311
Epoch 0 step 254: training accuarcy: 0.7397
Epoch 0 step 254: training loss: 7135.725203376929
Epoch 0 step 255: training accuarcy: 0.7260000000000001
Epoch 0 step 255: training loss: 7094.504852643588
Epoch 0 step 256: training accuarcy: 0.7266
Epoch 0 step 256: training loss: 7054.46104374724
Epoch 0 step 257: training accuarcy: 0.736
Epoch 0 step 257: training loss: 7040.11812783607
Epoch 0 step 258: training accuarcy: 0.736
Epoch 0 step 258: training loss: 7043.223295296297
Epoch 0 step 259: training accuarcy: 0.7363000000000001
Epoch 0 step 259: training loss: 7029.740564556023
Epoch 0 step 260: training accuarcy: 0.7281000000000001
Epoch 0 step 260: training loss: 6942.707432642393
Epoch 0 step 261: training accuarcy: 0.7453000000000001
Epoch 0

...

Epoch: 2
Epoch 2 step 526: training loss: 5959.227442539103
Epoch 2 step 527: training accuarcy: 0.766
Epoch 2 step 527: training loss: 5811.93729502977
Epoch 2 step 528: training accuarcy: 0.7775000000000001
Epoch 2 step 528: training loss: 5794.827736664424
Epoch 2 step 529: training accuarcy: 0.7781
Epoch 2 step 529: training loss: 5927.356809450854
Epoch 2 step 530: training accuarcy: 0.7644000000000001
Epoch 2 step 530: training loss: 5839.781489082489
Epoch 2 step 531: training accuarcy: 0.774
Epoch 2 step 531: training loss: 5938.867081760262
Epoch 2 step 532: training accuarcy: 0.7657
Epoch 2 step 532: training loss: 5874.202767021185
Epoch 2 step 533: training accuarcy: 0.7662
Epoch 2 step 533: training loss: 5907.050374286284
Epoch 2 step 534: training accuarcy: 0.7687
Epoch 2 step 534: training loss: 5866.3597528698665
Epoch 2 step 535: training accuarcy: 0.7724000000000001
Epoch 2 step 535: training loss: 5973.441046847933
Epoch 2 step 536: training accuarcy: 0.751400000000

Epoch 2 step 609: training loss: 5868.803261274316
Epoch 2 step 610: training accuarcy: 0.7607
Epoch 2 step 610: training loss: 5821.171576013072
Epoch 2 step 611: training accuarcy: 0.7678
Epoch 2 step 611: training loss: 5812.326536367214
Epoch 2 step 612: training accuarcy: 0.7585000000000001
Epoch 2 step 612: training loss: 5713.2006609790205
Epoch 2 step 613: training accuarcy: 0.7754000000000001
Epoch 2 step 613: training loss: 5947.645456339093
Epoch 2 step 614: training accuarcy: 0.7582
Epoch 2 step 614: training loss: 5871.072265700461
Epoch 2 step 615: training accuarcy: 0.7597
Epoch 2 step 615: training loss: 5869.44051157142
Epoch 2 step 616: training accuarcy: 0.756
Epoch 2 step 616: training loss: 5921.751201514423
Epoch 2 step 617: training accuarcy: 0.7582
Epoch 2 step 617: training loss: 5823.039565433438
Epoch 2 step 618: training accuarcy: 0.7719
Epoch 2 step 618: training loss: 5787.280896895166
Epoch 2 step 619: training accuarcy: 0.7732
Epoch 2 step 619: training 

Epoch 2 step 692: training loss: 5808.003758137652
Epoch 2 step 693: training accuarcy: 0.7599
Epoch 2 step 693: training loss: 5840.707691860372
Epoch 2 step 694: training accuarcy: 0.7573000000000001
Epoch 2 step 694: training loss: 5785.5937861688335
Epoch 2 step 695: training accuarcy: 0.7553000000000001
Epoch 2 step 695: training loss: 5733.99923466894
Epoch 2 step 696: training accuarcy: 0.771
Epoch 2 step 696: training loss: 5852.235513384204
Epoch 2 step 697: training accuarcy: 0.7562
Epoch 2 step 697: training loss: 5778.643419491904
Epoch 2 step 698: training accuarcy: 0.7635000000000001
Epoch 2 step 698: training loss: 5815.865021710468
Epoch 2 step 699: training accuarcy: 0.7537
Epoch 2 step 699: training loss: 5777.87393824913
Epoch 2 step 700: training accuarcy: 0.7724000000000001
Epoch 2 step 700: training loss: 5796.416080089128
Epoch 2 step 701: training accuarcy: 0.7594000000000001
Epoch 2 step 701: training loss: 5751.790553334917
Epoch 2 step 702: training accuarcy:

Epoch 2 step 774: training loss: 5791.259595682465
Epoch 2 step 775: training accuarcy: 0.7603000000000001
Epoch 2 step 775: training loss: 5745.034903773548
Epoch 2 step 776: training accuarcy: 0.7605000000000001
Epoch 2 step 776: training loss: 5731.012454404853
Epoch 2 step 777: training accuarcy: 0.7651
Epoch 2 step 777: training loss: 5750.86977893035
Epoch 2 step 778: training accuarcy: 0.7672
Epoch 2 step 778: training loss: 5734.934685328116
Epoch 2 step 779: training accuarcy: 0.7657
Epoch 2 step 779: training loss: 5780.4799475845475
Epoch 2 step 780: training accuarcy: 0.7595000000000001
Epoch 2 step 780: training loss: 5742.457617607512
Epoch 2 step 781: training accuarcy: 0.7624000000000001
Epoch 2 step 781: training loss: 5692.319438713783
Epoch 2 step 782: training accuarcy: 0.7679
Epoch 2 step 782: training loss: 5704.255661695401
Epoch 2 step 783: training accuarcy: 0.7632
Epoch 2 step 783: training loss: 5657.374682291392
Epoch 2 step 784: training accuarcy: 0.7682
Ep

100%|████████████████████████████████████████████| 3/3 [14:58<00:00, 299.88s/it]


In [41]:
prme_learner.fit(epoch=3,
                 log_dir=get_log_dir('weight_topcoder', 'prme'))

  0%|                                                     | 0/3 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 294606.13853030634
Epoch 0 step 1: training accuarcy: 0.5142
Epoch 0 step 1: training loss: 286108.444642681
Epoch 0 step 2: training accuarcy: 0.5282
Epoch 0 step 2: training loss: 293987.4134806083
Epoch 0 step 3: training accuarcy: 0.5083
Epoch 0 step 3: training loss: 284043.4721217331
Epoch 0 step 4: training accuarcy: 0.5192
Epoch 0 step 4: training loss: 278149.06834666454
Epoch 0 step 5: training accuarcy: 0.5146000000000001
Epoch 0 step 5: training loss: 279162.83377816185
Epoch 0 step 6: training accuarcy: 0.5137
Epoch 0 step 6: training loss: 263925.41081593785
Epoch 0 step 7: training accuarcy: 0.533
Epoch 0 step 7: training loss: 272980.8215130734
Epoch 0 step 8: training accuarcy: 0.5086
Epoch 0 step 8: training loss: 261422.55033331588
Epoch 0 step 9: training accuarcy: 0.5252
Epoch 0 step 9: training loss: 246522.51367903984
Epoch 0 step 10: training accuarcy: 0.5244
Epoch 0 step 10: training loss: 247796.70586116522
Epoch 0 step 

Epoch 0 step 86: training loss: 49351.753423952265
Epoch 0 step 87: training accuarcy: 0.5918
Epoch 0 step 87: training loss: 50024.61976459105
Epoch 0 step 88: training accuarcy: 0.5839
Epoch 0 step 88: training loss: 49202.58714893614
Epoch 0 step 89: training accuarcy: 0.5897
Epoch 0 step 89: training loss: 48909.41989555757
Epoch 0 step 90: training accuarcy: 0.5923
Epoch 0 step 90: training loss: 48638.808378756796
Epoch 0 step 91: training accuarcy: 0.5889
Epoch 0 step 91: training loss: 49006.547890239744
Epoch 0 step 92: training accuarcy: 0.5831000000000001
Epoch 0 step 92: training loss: 47676.21878923138
Epoch 0 step 93: training accuarcy: 0.6065
Epoch 0 step 93: training loss: 48449.83790047556
Epoch 0 step 94: training accuarcy: 0.5983
Epoch 0 step 94: training loss: 47852.71035297997
Epoch 0 step 95: training accuarcy: 0.6043000000000001
Epoch 0 step 95: training loss: 47793.16903701683
Epoch 0 step 96: training accuarcy: 0.6057
Epoch 0 step 96: training loss: 47773.42434

Epoch 0 step 170: training loss: 39202.0034520271
Epoch 0 step 171: training accuarcy: 0.6786
Epoch 0 step 171: training loss: 39170.801937912816
Epoch 0 step 172: training accuarcy: 0.6756
Epoch 0 step 172: training loss: 39557.2955702388
Epoch 0 step 173: training accuarcy: 0.669
Epoch 0 step 173: training loss: 39067.8986854975
Epoch 0 step 174: training accuarcy: 0.6762
Epoch 0 step 174: training loss: 38976.90380625868
Epoch 0 step 175: training accuarcy: 0.6856
Epoch 0 step 175: training loss: 39097.7987199493
Epoch 0 step 176: training accuarcy: 0.6751
Epoch 0 step 176: training loss: 39096.42293060083
Epoch 0 step 177: training accuarcy: 0.6756
Epoch 0 step 177: training loss: 38485.49366859559
Epoch 0 step 178: training accuarcy: 0.6910000000000001
Epoch 0 step 178: training loss: 39472.11341639771
Epoch 0 step 179: training accuarcy: 0.6763
Epoch 0 step 179: training loss: 39325.646085751294
Epoch 0 step 180: training accuarcy: 0.6801
Epoch 0 step 180: training loss: 39364.10

Epoch 0 step 253: training loss: 35283.5836472917
Epoch 0 step 254: training accuarcy: 0.7287
Epoch 0 step 254: training loss: 36261.26602595637
Epoch 0 step 255: training accuarcy: 0.7130000000000001
Epoch 0 step 255: training loss: 35349.418825336514
Epoch 0 step 256: training accuarcy: 0.7243
Epoch 0 step 256: training loss: 35795.27394716085
Epoch 0 step 257: training accuarcy: 0.7185
Epoch 0 step 257: training loss: 35761.73537550089
Epoch 0 step 258: training accuarcy: 0.7175
Epoch 0 step 258: training loss: 35705.99420170844
Epoch 0 step 259: training accuarcy: 0.7271000000000001
Epoch 0 step 259: training loss: 35823.21189438501
Epoch 0 step 260: training accuarcy: 0.7209
Epoch 0 step 260: training loss: 36391.402454865245
Epoch 0 step 261: training accuarcy: 0.7184
Epoch 0 step 261: training loss: 35469.310334724505
Epoch 0 step 262: training accuarcy: 0.7233
Epoch 0 step 262: training loss: 19016.696263744274
Epoch 0 step 263: training accuarcy: 0.7282051282051282
Epoch 0: tr

...

Epoch: 2
Epoch 2 step 526: training loss: 29592.70226770191
Epoch 2 step 527: training accuarcy: 0.7885000000000001
Epoch 2 step 527: training loss: 29542.998646134674
Epoch 2 step 528: training accuarcy: 0.7934
Epoch 2 step 528: training loss: 29294.765387400574
Epoch 2 step 529: training accuarcy: 0.7911
Epoch 2 step 529: training loss: 29082.70863222242
Epoch 2 step 530: training accuarcy: 0.7921
Epoch 2 step 530: training loss: 29819.621505648596
Epoch 2 step 531: training accuarcy: 0.7924
Epoch 2 step 531: training loss: 30067.853013730157
Epoch 2 step 532: training accuarcy: 0.781
Epoch 2 step 532: training loss: 29661.87719487754
Epoch 2 step 533: training accuarcy: 0.7889
Epoch 2 step 533: training loss: 29798.27668340026
Epoch 2 step 534: training accuarcy: 0.79
Epoch 2 step 534: training loss: 29361.29082672926
Epoch 2 step 535: training accuarcy: 0.7918000000000001
Epoch 2 step 535: training loss: 29210.004528644535
Epoch 2 step 536: training accuarcy: 0.8016000000000001
Epo

Epoch 2 step 609: training accuarcy: 0.7956000000000001
Epoch 2 step 609: training loss: 29664.6323343107
Epoch 2 step 610: training accuarcy: 0.7792
Epoch 2 step 610: training loss: 28915.086062798895
Epoch 2 step 611: training accuarcy: 0.8003
Epoch 2 step 611: training loss: 29135.636845866713
Epoch 2 step 612: training accuarcy: 0.7852
Epoch 2 step 612: training loss: 29208.651239100323
Epoch 2 step 613: training accuarcy: 0.7909
Epoch 2 step 613: training loss: 29833.65878955085
Epoch 2 step 614: training accuarcy: 0.7789
Epoch 2 step 614: training loss: 29102.917230686304
Epoch 2 step 615: training accuarcy: 0.7908000000000001
Epoch 2 step 615: training loss: 28425.888775653133
Epoch 2 step 616: training accuarcy: 0.7987000000000001
Epoch 2 step 616: training loss: 28327.93327843025
Epoch 2 step 617: training accuarcy: 0.7948000000000001
Epoch 2 step 617: training loss: 28958.10608098931
Epoch 2 step 618: training accuarcy: 0.7822
Epoch 2 step 618: training loss: 29738.3849679768

Epoch 2 step 690: training loss: 28686.99754215468
Epoch 2 step 691: training accuarcy: 0.7906000000000001
Epoch 2 step 691: training loss: 28763.448810794554
Epoch 2 step 692: training accuarcy: 0.7953
Epoch 2 step 692: training loss: 28734.754986791093
Epoch 2 step 693: training accuarcy: 0.7903
Epoch 2 step 693: training loss: 28832.191333561503
Epoch 2 step 694: training accuarcy: 0.7874
Epoch 2 step 694: training loss: 28889.438832097236
Epoch 2 step 695: training accuarcy: 0.7891
Epoch 2 step 695: training loss: 29524.77223648897
Epoch 2 step 696: training accuarcy: 0.7811
Epoch 2 step 696: training loss: 28111.733619486877
Epoch 2 step 697: training accuarcy: 0.793
Epoch 2 step 697: training loss: 28398.059954069533
Epoch 2 step 698: training accuarcy: 0.7891
Epoch 2 step 698: training loss: 28536.42865893506
Epoch 2 step 699: training accuarcy: 0.7935
Epoch 2 step 699: training loss: 28458.421778173768
Epoch 2 step 700: training accuarcy: 0.793
Epoch 2 step 700: training loss: 

Epoch 2 step 772: training accuarcy: 0.7816000000000001
Epoch 2 step 772: training loss: 28596.644890944728
Epoch 2 step 773: training accuarcy: 0.7939
Epoch 2 step 773: training loss: 28627.829615390678
Epoch 2 step 774: training accuarcy: 0.783
Epoch 2 step 774: training loss: 28547.175131956585
Epoch 2 step 775: training accuarcy: 0.7868
Epoch 2 step 775: training loss: 28239.733361198018
Epoch 2 step 776: training accuarcy: 0.7979
Epoch 2 step 776: training loss: 28080.279394928348
Epoch 2 step 777: training accuarcy: 0.798
Epoch 2 step 777: training loss: 28487.64659201348
Epoch 2 step 778: training accuarcy: 0.7872
Epoch 2 step 778: training loss: 27957.995743259737
Epoch 2 step 779: training accuarcy: 0.7943
Epoch 2 step 779: training loss: 28439.689447924073
Epoch 2 step 780: training accuarcy: 0.7892
Epoch 2 step 780: training loss: 28469.032599272432
Epoch 2 step 781: training accuarcy: 0.7901
Epoch 2 step 781: training loss: 28173.868287630146
Epoch 2 step 782: training accu

100%|████████████████████████████████████████████| 3/3 [14:33<00:00, 289.31s/it]


In [42]:
del prme_model
T.cuda.empty_cache()

### Train Trans FM Model

In [49]:
trans_model = TorchTransFM(feature_dim=feat_dim, num_dim=NUM_DIM, init_mean=INIT_MEAN)
trans_model

TorchTransFM()

In [50]:
adam_opt = optim.Adam(trans_model.parameters(), lr=LEARNING_RATE)
schedular = optim.lr_scheduler.StepLR(adam_opt,
                                      step_size=DECAY_FREQ,
                                      gamma=DECAY_GAMMA)

In [51]:
trans_learner = FMLearner(trans_model, adam_opt, schedular, db)
trans_learner

In [28]:
trans_learner.compile(train_col='base',
                      valid_col='seq',
                      test_col='seq',
                      loss_callback=trans_loss_callback)

In [46]:
trans_learner.compile(train_col='seq',
                      valid_col='seq',
                      test_col='seq',
                      loss_callback=trans_loss_callback)

In [52]:
trans_learner.compile(train_col='seq',
                      valid_col='seq',
                      test_col='seq',
                      loss_callback=trans_weight_loss_callback)

In [47]:
trans_learner.fit(epoch=3,
                  log_dir=get_log_dir('simple_topcoder', 'trans'))

  0%|                                                     | 0/3 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 90523.66535448213
Epoch 0 step 1: training accuarcy: 0.5174
Epoch 0 step 1: training loss: 87424.23827851049
Epoch 0 step 2: training accuarcy: 0.5308
Epoch 0 step 2: training loss: 86097.55888972402
Epoch 0 step 3: training accuarcy: 0.5161
Epoch 0 step 3: training loss: 87003.45660238003
Epoch 0 step 4: training accuarcy: 0.5036
Epoch 0 step 4: training loss: 82552.1749683306
Epoch 0 step 5: training accuarcy: 0.5275000000000001
Epoch 0 step 5: training loss: 80373.29348692777
Epoch 0 step 6: training accuarcy: 0.5214
Epoch 0 step 6: training loss: 78654.98681077069
Epoch 0 step 7: training accuarcy: 0.5228
Epoch 0 step 7: training loss: 76674.39022174261
Epoch 0 step 8: training accuarcy: 0.5243
Epoch 0 step 8: training loss: 74296.39480536147
Epoch 0 step 9: training accuarcy: 0.5236000000000001
Epoch 0 step 9: training loss: 74795.55691764891
Epoch 0 step 10: training accuarcy: 0.5092
Epoch 0 step 10: training loss: 71093.94043584414
Epoch 0

Epoch 0 step 85: training loss: 10060.416775912365
Epoch 0 step 86: training accuarcy: 0.7482000000000001
Epoch 0 step 86: training loss: 9865.594381436453
Epoch 0 step 87: training accuarcy: 0.7548
Epoch 0 step 87: training loss: 9613.980609779977
Epoch 0 step 88: training accuarcy: 0.7628
Epoch 0 step 88: training loss: 9493.501054391545
Epoch 0 step 89: training accuarcy: 0.7695000000000001
Epoch 0 step 89: training loss: 9559.720245734807
Epoch 0 step 90: training accuarcy: 0.7531
Epoch 0 step 90: training loss: 9471.13026649403
Epoch 0 step 91: training accuarcy: 0.7574000000000001
Epoch 0 step 91: training loss: 9249.999336803165
Epoch 0 step 92: training accuarcy: 0.7632
Epoch 0 step 92: training loss: 9105.271713042825
Epoch 0 step 93: training accuarcy: 0.7708
Epoch 0 step 93: training loss: 8990.013228916067
Epoch 0 step 94: training accuarcy: 0.7721
Epoch 0 step 94: training loss: 8950.004093611804
Epoch 0 step 95: training accuarcy: 0.7605000000000001
Epoch 0 step 95: train

Epoch 0 step 167: training loss: 5210.732623835984
Epoch 0 step 168: training accuarcy: 0.8262
Epoch 0 step 168: training loss: 5251.075820998959
Epoch 0 step 169: training accuarcy: 0.8245
Epoch 0 step 169: training loss: 5096.3778992297775
Epoch 0 step 170: training accuarcy: 0.8371000000000001
Epoch 0 step 170: training loss: 5021.562767639163
Epoch 0 step 171: training accuarcy: 0.8380000000000001
Epoch 0 step 171: training loss: 5114.859250081069
Epoch 0 step 172: training accuarcy: 0.8324
Epoch 0 step 172: training loss: 4982.526724791125
Epoch 0 step 173: training accuarcy: 0.8390000000000001
Epoch 0 step 173: training loss: 5186.525137789706
Epoch 0 step 174: training accuarcy: 0.8232
Epoch 0 step 174: training loss: 4994.231141879069
Epoch 0 step 175: training accuarcy: 0.8330000000000001
Epoch 0 step 175: training loss: 4846.676476891661
Epoch 0 step 176: training accuarcy: 0.8450000000000001
Epoch 0 step 176: training loss: 4917.665380151637
Epoch 0 step 177: training accuar

Epoch 0 step 249: training accuarcy: 0.8744000000000001
Epoch 0 step 249: training loss: 3999.6635735679865
Epoch 0 step 250: training accuarcy: 0.8548
Epoch 0 step 250: training loss: 3790.502622343316
Epoch 0 step 251: training accuarcy: 0.8615
Epoch 0 step 251: training loss: 3842.599576680568
Epoch 0 step 252: training accuarcy: 0.8667
Epoch 0 step 252: training loss: 3826.4443657936545
Epoch 0 step 253: training accuarcy: 0.8674000000000001
Epoch 0 step 253: training loss: 3763.066579905333
Epoch 0 step 254: training accuarcy: 0.8676
Epoch 0 step 254: training loss: 3776.3742291766503
Epoch 0 step 255: training accuarcy: 0.8646
Epoch 0 step 255: training loss: 3753.2273333305825
Epoch 0 step 256: training accuarcy: 0.8706
Epoch 0 step 256: training loss: 3712.96122234752
Epoch 0 step 257: training accuarcy: 0.8706
Epoch 0 step 257: training loss: 3782.381695555322
Epoch 0 step 258: training accuarcy: 0.8635
Epoch 0 step 258: training loss: 3722.9243131356257
Epoch 0 step 259: trai

...

Epoch: 2
Epoch 2 step 526: training loss: 2463.249562397879
Epoch 2 step 527: training accuarcy: 0.9244
Epoch 2 step 527: training loss: 2644.7975321448034
Epoch 2 step 528: training accuarcy: 0.9162
Epoch 2 step 528: training loss: 2526.4155175960937
Epoch 2 step 529: training accuarcy: 0.9206000000000001
Epoch 2 step 529: training loss: 2498.493884063634
Epoch 2 step 530: training accuarcy: 0.9211
Epoch 2 step 530: training loss: 2561.152050748146
Epoch 2 step 531: training accuarcy: 0.9204
Epoch 2 step 531: training loss: 2576.504245167436
Epoch 2 step 532: training accuarcy: 0.916
Epoch 2 step 532: training loss: 2489.975740599931
Epoch 2 step 533: training accuarcy: 0.9268000000000001
Epoch 2 step 533: training loss: 2462.0078116794516
Epoch 2 step 534: training accuarcy: 0.9252
Epoch 2 step 534: training loss: 2488.068505668272
Epoch 2 step 535: training accuarcy: 0.9231
Epoch 2 step 535: training loss: 2501.043814013624
Epoch 2 step 536: training accuarcy: 0.9212
Epoch 2 step 53

Epoch 2 step 608: training accuarcy: 0.9246000000000001
Epoch 2 step 608: training loss: 2520.2222590527595
Epoch 2 step 609: training accuarcy: 0.9197000000000001
Epoch 2 step 609: training loss: 2367.705036605018
Epoch 2 step 610: training accuarcy: 0.9271
Epoch 2 step 610: training loss: 2434.760432207689
Epoch 2 step 611: training accuarcy: 0.92
Epoch 2 step 611: training loss: 2414.168806405387
Epoch 2 step 612: training accuarcy: 0.9228000000000001
Epoch 2 step 612: training loss: 2496.5671979814115
Epoch 2 step 613: training accuarcy: 0.9201
Epoch 2 step 613: training loss: 2409.4713203134415
Epoch 2 step 614: training accuarcy: 0.9246000000000001
Epoch 2 step 614: training loss: 2498.5908265848598
Epoch 2 step 615: training accuarcy: 0.9163
Epoch 2 step 615: training loss: 2547.7652123532403
Epoch 2 step 616: training accuarcy: 0.9169
Epoch 2 step 616: training loss: 2497.6139012080275
Epoch 2 step 617: training accuarcy: 0.9164
Epoch 2 step 617: training loss: 2562.8692561907


Epoch 2 step 691: training accuarcy: 0.9211
Epoch 2 step 691: training loss: 2422.6347955982683
Epoch 2 step 692: training accuarcy: 0.9218000000000001
Epoch 2 step 692: training loss: 2428.002158131525
Epoch 2 step 693: training accuarcy: 0.9197000000000001
Epoch 2 step 693: training loss: 2430.978688118582
Epoch 2 step 694: training accuarcy: 0.92
Epoch 2 step 694: training loss: 2399.291982053827
Epoch 2 step 695: training accuarcy: 0.9246000000000001
Epoch 2 step 695: training loss: 2494.235013117953
Epoch 2 step 696: training accuarcy: 0.9153
Epoch 2 step 696: training loss: 2473.1082577662205
Epoch 2 step 697: training accuarcy: 0.9198000000000001
Epoch 2 step 697: training loss: 2384.8626362919535
Epoch 2 step 698: training accuarcy: 0.9259000000000001
Epoch 2 step 698: training loss: 2390.7218758842905
Epoch 2 step 699: training accuarcy: 0.9244
Epoch 2 step 699: training loss: 2475.6005071662676
Epoch 2 step 700: training accuarcy: 0.9165000000000001
Epoch 2 step 700: training

Epoch 2 step 773: training accuarcy: 0.9269000000000001
Epoch 2 step 773: training loss: 2413.9861168234124
Epoch 2 step 774: training accuarcy: 0.9197000000000001
Epoch 2 step 774: training loss: 2418.5103837021084
Epoch 2 step 775: training accuarcy: 0.9178000000000001
Epoch 2 step 775: training loss: 2332.8443987128853
Epoch 2 step 776: training accuarcy: 0.9243
Epoch 2 step 776: training loss: 2422.3543618005665
Epoch 2 step 777: training accuarcy: 0.9184
Epoch 2 step 777: training loss: 2449.8599825882125
Epoch 2 step 778: training accuarcy: 0.9159
Epoch 2 step 778: training loss: 2331.0916550110114
Epoch 2 step 779: training accuarcy: 0.9268000000000001
Epoch 2 step 779: training loss: 2346.5691548063824
Epoch 2 step 780: training accuarcy: 0.9271
Epoch 2 step 780: training loss: 2295.2918773760234
Epoch 2 step 781: training accuarcy: 0.9276000000000001
Epoch 2 step 781: training loss: 2292.7681471379606
Epoch 2 step 782: training accuarcy: 0.9297000000000001
Epoch 2 step 782: tr

100%|████████████████████████████████████████████| 3/3 [14:55<00:00, 299.24s/it]


In [53]:
trans_learner.fit(epoch=3,
                  log_dir=get_log_dir('weight_topcoder', 'trans'))

  0%|                                                     | 0/3 [00:00<?, ?it/s]

Epoch: 0
Epoch 0 step 0: training loss: 328693.0835166554
Epoch 0 step 1: training accuarcy: 0.4978
Epoch 0 step 1: training loss: 314058.7249141159
Epoch 0 step 2: training accuarcy: 0.5168
Epoch 0 step 2: training loss: 300176.1108415055
Epoch 0 step 3: training accuarcy: 0.5159
Epoch 0 step 3: training loss: 307014.09675405966
Epoch 0 step 4: training accuarcy: 0.5114000000000001
Epoch 0 step 4: training loss: 297557.54419490095
Epoch 0 step 5: training accuarcy: 0.5051
Epoch 0 step 5: training loss: 291071.6806946936
Epoch 0 step 6: training accuarcy: 0.5202
Epoch 0 step 6: training loss: 281828.43845424784
Epoch 0 step 7: training accuarcy: 0.5189
Epoch 0 step 7: training loss: 268520.46315127116
Epoch 0 step 8: training accuarcy: 0.5279
Epoch 0 step 8: training loss: 272973.084331861
Epoch 0 step 9: training accuarcy: 0.5239
Epoch 0 step 9: training loss: 262233.4747713839
Epoch 0 step 10: training accuarcy: 0.5155000000000001
Epoch 0 step 10: training loss: 253659.75581515126
Ep

Epoch 0 step 84: training loss: 37021.59833738739
Epoch 0 step 85: training accuarcy: 0.7614000000000001
Epoch 0 step 85: training loss: 37115.563093654804
Epoch 0 step 86: training accuarcy: 0.7664000000000001
Epoch 0 step 86: training loss: 36621.16092949527
Epoch 0 step 87: training accuarcy: 0.7632
Epoch 0 step 87: training loss: 36754.84782784991
Epoch 0 step 88: training accuarcy: 0.7665000000000001
Epoch 0 step 88: training loss: 37244.81074448023
Epoch 0 step 89: training accuarcy: 0.7556
Epoch 0 step 89: training loss: 36105.546669099494
Epoch 0 step 90: training accuarcy: 0.7735000000000001
Epoch 0 step 90: training loss: 36084.91944435393
Epoch 0 step 91: training accuarcy: 0.7739
Epoch 0 step 91: training loss: 35836.71114717996
Epoch 0 step 92: training accuarcy: 0.7769
Epoch 0 step 92: training loss: 35005.81801922584
Epoch 0 step 93: training accuarcy: 0.7817000000000001
Epoch 0 step 93: training loss: 35921.09514532122
Epoch 0 step 94: training accuarcy: 0.7675000000000

Epoch 0 step 166: training loss: 26601.093323936373
Epoch 0 step 167: training accuarcy: 0.8469
Epoch 0 step 167: training loss: 26099.55135480284
Epoch 0 step 168: training accuarcy: 0.8402000000000001
Epoch 0 step 168: training loss: 25822.06086071969
Epoch 0 step 169: training accuarcy: 0.8493
Epoch 0 step 169: training loss: 26762.53642552209
Epoch 0 step 170: training accuarcy: 0.8391000000000001
Epoch 0 step 170: training loss: 25803.678595341225
Epoch 0 step 171: training accuarcy: 0.8464
Epoch 0 step 171: training loss: 25582.729381492667
Epoch 0 step 172: training accuarcy: 0.8496
Epoch 0 step 172: training loss: 26239.21400536701
Epoch 0 step 173: training accuarcy: 0.8427
Epoch 0 step 173: training loss: 25087.77036797405
Epoch 0 step 174: training accuarcy: 0.8572000000000001
Epoch 0 step 174: training loss: 25512.108946995184
Epoch 0 step 175: training accuarcy: 0.8507
Epoch 0 step 175: training loss: 25481.058037329185
Epoch 0 step 176: training accuarcy: 0.85030000000000

Epoch 0 step 247: training loss: 21181.59605143062
Epoch 0 step 248: training accuarcy: 0.8744000000000001
Epoch 0 step 248: training loss: 20717.414362144315
Epoch 0 step 249: training accuarcy: 0.8818
Epoch 0 step 249: training loss: 21020.817223240316
Epoch 0 step 250: training accuarcy: 0.8737
Epoch 0 step 250: training loss: 20807.829010201334
Epoch 0 step 251: training accuarcy: 0.8784000000000001
Epoch 0 step 251: training loss: 20442.383027732518
Epoch 0 step 252: training accuarcy: 0.8804000000000001
Epoch 0 step 252: training loss: 20480.054775489694
Epoch 0 step 253: training accuarcy: 0.8854000000000001
Epoch 0 step 253: training loss: 20596.99193253302
Epoch 0 step 254: training accuarcy: 0.8804000000000001
Epoch 0 step 254: training loss: 20183.7893707749
Epoch 0 step 255: training accuarcy: 0.8834000000000001
Epoch 0 step 255: training loss: 20341.919353814716
Epoch 0 step 256: training accuarcy: 0.8811
Epoch 0 step 256: training loss: 20605.72333395158
Epoch 0 step 257:

...

Epoch: 2
Epoch 2 step 526: training loss: 12579.079971786003
Epoch 2 step 527: training accuarcy: 0.932
Epoch 2 step 527: training loss: 12467.619066556137
Epoch 2 step 528: training accuarcy: 0.9273
Epoch 2 step 528: training loss: 12956.344311152803
Epoch 2 step 529: training accuarcy: 0.9261
Epoch 2 step 529: training loss: 12257.875309358122
Epoch 2 step 530: training accuarcy: 0.9339000000000001
Epoch 2 step 530: training loss: 12327.81546683406
Epoch 2 step 531: training accuarcy: 0.934
Epoch 2 step 531: training loss: 12897.246040983051
Epoch 2 step 532: training accuarcy: 0.9271
Epoch 2 step 532: training loss: 12555.067693549696
Epoch 2 step 533: training accuarcy: 0.9292
Epoch 2 step 533: training loss: 12355.227634627197
Epoch 2 step 534: training accuarcy: 0.9334
Epoch 2 step 534: training loss: 12954.758991505158
Epoch 2 step 535: training accuarcy: 0.9284
Epoch 2 step 535: training loss: 12456.148345488487
Epoch 2 step 536: training accuarcy: 0.9321
Epoch 2 step 536: trai

Epoch 2 step 609: training loss: 12326.172528546733
Epoch 2 step 610: training accuarcy: 0.9286000000000001
Epoch 2 step 610: training loss: 11747.185809195607
Epoch 2 step 611: training accuarcy: 0.9336000000000001
Epoch 2 step 611: training loss: 12297.782410734995
Epoch 2 step 612: training accuarcy: 0.9331
Epoch 2 step 612: training loss: 11963.905872445495
Epoch 2 step 613: training accuarcy: 0.9299000000000001
Epoch 2 step 613: training loss: 12053.08268501908
Epoch 2 step 614: training accuarcy: 0.9276000000000001
Epoch 2 step 614: training loss: 12080.945894977689
Epoch 2 step 615: training accuarcy: 0.9289000000000001
Epoch 2 step 615: training loss: 12388.82632281529
Epoch 2 step 616: training accuarcy: 0.9233
Epoch 2 step 616: training loss: 12094.645916003059
Epoch 2 step 617: training accuarcy: 0.9303
Epoch 2 step 617: training loss: 12037.060737366077
Epoch 2 step 618: training accuarcy: 0.9322
Epoch 2 step 618: training loss: 12039.309792153434
Epoch 2 step 619: training

Epoch 2 step 690: training accuarcy: 0.9318000000000001
Epoch 2 step 690: training loss: 11941.249305893562
Epoch 2 step 691: training accuarcy: 0.9281
Epoch 2 step 691: training loss: 11631.430190287456
Epoch 2 step 692: training accuarcy: 0.9298000000000001
Epoch 2 step 692: training loss: 11589.049984246412
Epoch 2 step 693: training accuarcy: 0.9275
Epoch 2 step 693: training loss: 11455.207522423885
Epoch 2 step 694: training accuarcy: 0.9323
Epoch 2 step 694: training loss: 11613.57102550763
Epoch 2 step 695: training accuarcy: 0.9311
Epoch 2 step 695: training loss: 11602.339899947161
Epoch 2 step 696: training accuarcy: 0.9281
Epoch 2 step 696: training loss: 11640.08287377311
Epoch 2 step 697: training accuarcy: 0.9309000000000001
Epoch 2 step 697: training loss: 11676.68072996902
Epoch 2 step 698: training accuarcy: 0.9277000000000001
Epoch 2 step 698: training loss: 11423.882087064068
Epoch 2 step 699: training accuarcy: 0.9314
Epoch 2 step 699: training loss: 11673.40305939

Epoch 2 step 772: training accuarcy: 0.9314
Epoch 2 step 772: training loss: 11156.548918593204
Epoch 2 step 773: training accuarcy: 0.9317000000000001
Epoch 2 step 773: training loss: 10960.9980655112
Epoch 2 step 774: training accuarcy: 0.9326000000000001
Epoch 2 step 774: training loss: 10597.352543006278
Epoch 2 step 775: training accuarcy: 0.936
Epoch 2 step 775: training loss: 10752.717864063485
Epoch 2 step 776: training accuarcy: 0.9375
Epoch 2 step 776: training loss: 10911.885200742496
Epoch 2 step 777: training accuarcy: 0.9334
Epoch 2 step 777: training loss: 11166.66684664055
Epoch 2 step 778: training accuarcy: 0.9295
Epoch 2 step 778: training loss: 10970.034636902245
Epoch 2 step 779: training accuarcy: 0.9328000000000001
Epoch 2 step 779: training loss: 10721.367067623518
Epoch 2 step 780: training accuarcy: 0.9366000000000001
Epoch 2 step 780: training loss: 11074.548780542173
Epoch 2 step 781: training accuarcy: 0.9321
Epoch 2 step 781: training loss: 11194.387461613

100%|████████████████████████████████████████████| 3/3 [15:24<00:00, 310.22s/it]


In [48]:
del trans_model
T.cuda.empty_cache()